In [1]:
# import requests
import requests
import json

In [12]:
# function to lookup lat and lon from address

def get_long_lat(address):
    url = f"https://nominatim.openstreetmap.org/search?q={address}&format=json"
    response = requests.get(url)
    data = response.json()
    return data[0]["lat"], data[0]["lon"]

get_long_lat("St. Leonhard-Strasse 45, St. Gallen")

('47.4217508', '9.369448')

In [14]:
# function to get heating info from long and lat

def get_heating_info(lat, lon, dist):
    url = f"https://daten.stadt.sg.ch/api/records/1.0/search/?dataset=warmeversorgung&q=&facet=waermevers&geofilter.distance={lat}%2C{lon}%2C{dist}"
    response = requests.get(url)
    data = response.json()
    return data

# function to get heating info from address

def get_heating_info_from_address(address, dist):
    lat, lon = get_long_lat(address)
    return get_heating_info(lat, lon, dist)

get_heating_info_from_address("St. Leonhard-Strasse 45, St. Gallen", 1)

{'nhits': 1,
 'parameters': {'dataset': 'warmeversorgung',
  'rows': 10,
  'start': 0,
  'facet': ['waermevers'],
  'format': 'json',
  'geofilter.distance': ['47.4217508,9.369448,1'],
  'timezone': 'UTC'},
 'records': [{'datasetid': 'warmeversorgung',
   'recordid': '41a0060d5f24a87f0c0f390b4e363916bd084a21',
   'fields': {'geo_shape': {'coordinates': [[[9.369402092347727,
        47.42141803157732],
       [9.36880080458217, 47.421178234079115],
       [9.36845445807961, 47.42153135919113],
       [9.369074349371125, 47.421789138301264],
       [9.369639976357611, 47.42203562120369],
       [9.369902089204528, 47.42170847062191],
       [9.369460130185418, 47.421532842160765],
       [9.369402092347727, 47.42141803157732]]],
     'type': 'Polygon'},
    'waermevers': 'Fernwärme Ergänzungsgebiet',
    'geo_point_2d': [47.42160099351865, 9.369180665597083],
    'dist': '26.117715156181355'},
   'geometry': {'type': 'Point',
    'coordinates': [9.369180665597083, 47.42160099351865]},
  